#Travel Insurance

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

%cd '/content/drive/Shareddrives/space/colab-content/travel-insurance'

Mounted at /content/drive/
/content/drive/Shareddrives/space/colab-content/travel-insurance


In [ ]:
# importing rquired libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.manifold import TSNE
from progressbar import ProgressBar


In [3]:
data = pd.read_csv('travel insurance.csv')
data

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,NaN,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,NaN,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,NaN,41
...,...,...,...,...,...,...,...,...,...,...,...
63321,JZI,Airlines,Online,Basic Plan,No,111,JAPAN,35.0,12.25,M,31
63322,JZI,Airlines,Online,Basic Plan,No,58,CHINA,40.0,14.00,F,40
63323,JZI,Airlines,Online,Basic Plan,No,2,MALAYSIA,18.0,6.30,M,57
63324,JZI,Airlines,Online,Basic Plan,No,3,VIET NAM,18.0,6.30,M,63


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63326 entries, 0 to 63325
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Agency                63326 non-null  object 
 1   Agency Type           63326 non-null  object 
 2   Distribution Channel  63326 non-null  object 
 3   Product Name          63326 non-null  object 
 4   Claim                 63326 non-null  object 
 5   Duration              63326 non-null  int64  
 6   Destination           63326 non-null  object 
 7   Net Sales             63326 non-null  float64
 8   Commision (in value)  63326 non-null  float64
 9   Gender                18219 non-null  object 
 10  Age                   63326 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 5.3+ MB


In [5]:
X = data.drop(columns= 'Claim')
y = data['Claim']

print(f'shape of X: {X.shape}')
print(f'shape of y: {y.shape}')


shape of X: (63326, 10)
shape of y: (63326,)


## Data Cleaning

In [6]:
X.isnull().mean() * 100

Agency                   0.000000
Agency Type              0.000000
Distribution Channel     0.000000
Product Name             0.000000
Duration                 0.000000
Destination              0.000000
Net Sales                0.000000
Commision (in value)     0.000000
Gender                  71.229827
Age                      0.000000
dtype: float64

Dropping the column `Gender` since 71.2% of the data is missing and on the rest of the columns there is no missing data

In [7]:
X.drop(columns= 'Gender', inplace= True)
print(f'shape of X: {X.shape}')

shape of X: (63326, 9)


In [8]:
# unique elements in categorical variables
X_categorical = X.select_dtypes(include= ['object'])
X_categorical.nunique()

Agency                   16
Agency Type               2
Distribution Channel      2
Product Name             26
Destination             149
dtype: int64

In [9]:
destination_value_counts = X['Destination'].value_counts()
fig = px.bar(x = destination_value_counts.index, y = destination_value_counts.values)

fig.show()

Here for the destination most of the data is present on 20 coutries and rest on the reamaining. So we can do a one hot encoding in a way to represent only this top 20 countrires.

In [10]:
def top_n_one_hot_encoding(data, column, n):
    unique_values = data[column].unique()
    top_n_values = data[column].value_counts()[:n]
    encoded_data = pd.get_dummies(data[column], prefix= column)
    encoded_top_n_column_names= column+'_'+top_n_values.index.values
    return encoded_data[encoded_top_n_column_names]


destination_data_encoded = top_n_one_hot_encoding(X, 'Destination', 20)

on the rest of the data applying simple one hot encoding would be suffice.

In [11]:
# encoding non destination categorical featues and merging the df with encoded destination df

categorical_data_encoded = pd.get_dummies(X_categorical.drop(columns= 'Destination'))
categorical_data_encoded = pd.concat([categorical_data_encoded, destination_data_encoded], axis= 1)

In [12]:
# concatination numeric and encoded non-numeric dataframes

X = pd.concat([X.select_dtypes(exclude= ['object']),categorical_data_encoded], axis= 1 )
print(f'Final shape of X after data cleaning: {X.shape}')

Final shape of X after data cleaning: (63326, 70)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state= 42)

print(f'size of X_trian: {X_train.shape}')
print(f'size of y_trian: {y_train.shape}')
print(f'size of X_test: {X_test.shape}')
print(f'size of y_test: {y_test.shape}')

size of X_trian: (50660, 70)
size of y_trian: (50660,)
size of X_test: (12666, 70)
size of y_test: (12666,)


In [14]:
normalizer = Normalizer()

X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

## t-SNE

In [31]:
# Create a TSNE model
model = TSNE(n_components=2, perplexity=30, random_state=0, bar= ProgressBar())

# Fit the model to the data
model.fit(X)

# Transform the data to 2D
transformed_data = model.transform(X)


NameError: ignored

In [ ]:

# Plot the data
plt.scatter(transformed_data[:, 0], transformed_data[:, 1], c=y)
plt.show()

## KNN model

In [ ]:

param_grid = {
    'n_neighbors': range(1, 99, 2)
}

clf = KNeighborsClassifier()

grid = GridSearchCV(clf, param_grid, scoring= 'accuracy', cv= 5, n_jobs= -1)

grid.fit(X_train, y_train)

cv_results = grid.cv_results_


fig = px.line(x= cv_results['param_n_neighbors'], y= cv_results['mean_test_score'],
              labels={'x': 'K-value', 'y': 'Accuracy'}, title= 'K vs Accuracy')

fig.show()

In [ ]:
# building classifier with best K

clf = KNeighborsClassifier(n_neighbors= 11)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.9857887257224064

In [ ]:
confusion_matrix(y_test, y_pred)

array([[12486,     0],
       [  180,     0]])

## Naive Bayes model

In [18]:
from sklearn.naive_bayes import GaussianNB

In [26]:
clf = GaussianNB(var_smoothing= 0.0001)

clf.fit(X_train, y_train)


GaussianNB(var_smoothing=0.0001)

In [27]:
train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)

In [28]:
accuracy_score(y_test, test_predictions)

0.7165640296857729

In [29]:
confusion_matrix(y_test, test_predictions)

array([[8935, 3551],
       [  39,  141]])